In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

Mounted at /content/drive


1.   Van a recibir dos archivos con datos: train.csv y test4alumnxs.csv, los cuales contienen información sobre accidentes de tránsito en EEUU (más información abajo). La diferencia entre los archivos es que train.csv tiene la columna 'Severity' (severidad) mientras que test4alumnxs.csv no la tiene.

2.   El objetivo es usar los datos para construir un modelo que prediga la severidad del accidente (columna 'Severity', donde 0 es baja severidad y 1 es alta severidad). Para esto, primero pueden usar los datos del archivo train.csv para ajustar los parámetros de su modelo, ajustar los hiperparámetros, combinar los features para agregar nuevos, estandarizar los datos, etc. Al terminar esto, tienen un modelo entrenado en estos datos, y una idea de que tan bien funciona.


3.   Luego, aplican ese modelo a los datos de test4alumnxs.csv y generan un vector de probabilidades que representan la probabilidad de que cada accidente sea severo.

4.   Nosotros tenemos acceso a las etiquetas, así que podemos calcular el AUC de sus predicciones. Con eso construimos un ranking de los mejores clasificadores.

Los datos corresponden a registros de accidentes de tránsito en EEUU entre 2016 y 2021.

Las primeras dos columnas ("unnamed") no sirve nara nada y se pueden descartar.

La columna 'Severity' contiene la severidad del accidente (0 significa baja, 1 significa alta)

Luego tenemos la hora de comienzo y final del accidente, la latitud/longitud inicial y final del accidente (entre que valores queda delimitado), la distancia de camino afectada por el accidente, una descripción verbal hecha por un humano, número y nombre de la calle donde ocurrió, lado de la calle (izquierda o derecha), ciudad, condado, estado, código postal, huso horario, código de aeropuerto donde se encuentra la estación meterológica más cercana al accidente, hora en la cual se midió el clima,temperatura, temperatura del viento, humedad, presión, visibilidad, dirección del viento, condición climática (despejado, sol, nieve, etc), presencia de comodidades ("amenities") en la cercanía del accidente, presencia de loma de burro, presencia de un cruce, presencia de señal de ceder paso, presencia de unión de calles, presencia de cartel de no salida, presencia de vías de tren, presencia de rotonda, presencia de estación, presencia de signo de parar, presencia de moderadores de tráfico, presencia de señales de tránsito, presencia de calle en forma de U ("loop"), día u oscuridad de acuerdo a la salida del sol, día u oscuridad de acuerdo a la penumbra civil (si es o no necesario utilizar alumbramiento eléctrico), día u oscuridad de acuerdo a la penumbra náutica, día u oscuridad de acuerdo a la penumbra astronómica.

No todos estos campos son necesariamente útiles, y tampoco no todos están en un formato numérico directamente utilizable (por ejemplo, hay strings en la parte de descripción del accidente). Es parte del trabajo decidir que features quedarse, que features sumar, y como extraer información relevante de los features más complejos.

In [10]:
!git clone https://github.com/P11-P11/Data-Science.git

%cd Data-Science

!git checkout Clasification

Cloning into 'Data-Science'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 7.01 MiB | 5.00 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [15]:
import os

data_folder_path = 'Data'
files_in_data_folder = os.listdir(data_folder_path)

# Imprimir los nombres de los archivos
for file_name in files_in_data_folder:
    print(file_name)

dataframes = []

for file_name in os.listdir(data_folder_path):
    file_path = os.path.join(data_folder_path, file_name)

    if file_name.endswith('.csv'):
        df = pd.read_csv(file_path)
        dataframes.append(df)

train.csv
test4alumnxs.csv


In [ ]:
df_train = dataframes[0]
df_test = dataframes[1]

In [2]:
#SEVEROS
def rutaCerrada(x):
  if 'Road closed' in x:
    return 1
  else: return 0
def carrilesCerrados(x):
  if 'All lanes closed' in x:
    return 1
  else: return 0
#NO SEVEROS
def estadoSeguro(x):
  if x == 'CA' or x == 'FL':
    return 1
  else: return 0
def driveCaution(x):
  if 'Drive with caution' in x:
    return 1
  else: return 0
def traficoEstacionario(x):
  if 'Stationary traffic' in x:
    return 1
  else: return 0
def traficoLento(x):
  if 'Slow traffic' in x:
    return 1
  else: return 0



In [ ]:

df = df_train[['Description','State','Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition','Bump', 'Crossing','Junction','Railway','Roundabout','Stop','Turning_Loop','Sunrise_Sunset']]
#df['diaDeSemana'] = df[]
df['estadoSeguro'] = df['State'].apply(estadoSeguro)
df['driveCaution'] = df['Description'].apply(driveCaution)
df['traficoEstacionario'] = df['Description'].apply(traficoEstacionario)
df['traficoLento'] = df['Description'].apply(traficoLento)
df['rutaCerrada'] = df['Description'].apply(rutaCerrada)
df['carrilesCerrados'] = df['Description'].apply(carrilesCerrados)


In [5]:
y = np.array(df_train['Severity']) # armo el vector de etiquetas
# extraigo la matriz de features X
features = ['Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Speed(mph)', 'Precipitation(in)',  'Bump',
       'Crossing', 'Junction', 'Railway', 'Roundabout', 'Stop', 'Turning_Loop',
       'estadoSeguro', 'driveCaution', 'traficoEstacionario',
       'traficoLento', 'rutaCerrada', 'carrilesCerrados']
X = np.array(df[features])


In [6]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)



# pruebo con un random forest así como viene de fábrica
clf = RandomForestClassifier(1000)

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
# lo que tengo entregar
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_test, roc_train


(0.9750880038600959, 0.9999757869110867)